<a href="https://colab.research.google.com/github/jimenabarrerau/Act7/blob/main/Act7_A01706474.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

In [29]:
df=pd.read_csv('cuentas_credicel.csv', encoding = 'latin')
df.sample(10)

<ipython-input-29-e883e98029ae>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('cuentas_credicel.csv', encoding = 'latin')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
6466,6601,EJSFVJKU,24659,04/08/22 19:48,Samsung,A 03 Core,26S,3299.00,521.0,0.0,...,15.79,10000,37,NaN,55.0,PUEBLA,PUE.,Heroica Puebla De Zaragoza,Puebla,HEAT670422MPLRLR07
10477,10629,WTI8ZKRD,43104,14/11/22 14:06,NOKIA,C20 2+32GB AZUL B,13S,2689.83,795.0,0.0,...,29.56,3500,13,39715,43.0,ACAPULCO DE JUAREZ,GRO,Acapulco De Juarez,Guerrero,AAEE790726HGRLGN06
290,298,4PD5RJI9,1300,10/01/22 13:42,LG,K62 B AZUL,26S,4900.86,1225.0,0.0,...,25.00,7000,66,NaN,60.0,TLANCHINOL. HGO.,TLANCHINOL. HGO.,Tantoyuca,Veracruz,LUHS611030HHGCRN07
20961,21150,B4TET8O4,88242,26/03/23 13:46,MOTOROLA,MOTO G30 4+128GB GRIS B,13S,4672.86,1000.0,0.0,...,21.40,4750,13,93072,34.0,COXQUIHUI,VER,Papantla,Veracruz,PECE880701MVZRRL09
17606,17780,K9XT7R5C,72683,11/02/23 16:25,MOTOROLA,4G XT2239-17 E22I Gris Kit,26S,3508.83,685.0,0.0,...,19.52,7000,9,74373,31.0,HUAQUECHULA,PUE,Atlixco,Puebla,PAHA910829HPLLTN04
16243,16416,TGV4CBRI,63985,24/01/23 11:36,SAMSUNG,4G A035M GXY A03 3+32GB AZUL KIT,26S,2923.83,829.0,0.0,...,28.35,3500,12,72229,30.0,PUEBLA,PUE,Heroica Puebla De Zaragoza,Puebla,GAVB920509MPLLLL01
4764,4899,G8Q8FA9W,18773,04/07/22 15:20,SAMSUNG,LTE SM-A525M GXY A52 Azul,26S,8173.91,1349.0,0.0,...,16.50,7000,41,NaN,22.0,PUEBLA,PUE.,Heroica Puebla De Zaragoza,Puebla,AERS000331HVZNYLA3
10778,10930,TSKEC3TP,44735,19/11/22 11:01,SAMSUNG,4G A035M GXY A03 3+32GB NEGRO KIT,13S,3508.83,845.0,200.0,...,24.08,5500,13,93438,46.0,PAPANTLA,VER,Papantla,Veracruz,MESV760214HVZNNL02
9554,9703,DMMD00M2,39074,27/10/22 20:03,ZTE,Blade A3 2020,39S,2549.00,606.0,0.0,...,23.77,6000,25,29960,61.0,PALENQUE,CHIS,Emiliano Zapata,Tabasco,CATL611010HCSHRS02
5705,5840,7GFHPAI9,22025,21/07/22 19:16,ZTE,BLADE A3 2020 Gris,13S,2004.82,571.0,0.0,...,28.48,5000,13,NaN,38.0,ID,ID,Zacatlan,Puebla,AUVJ840302HPLGLR09


In [30]:
df['limite_credito'] = pd.to_numeric(df['limite_credito'], errors='coerce')
mediana = df['limite_credito'].median()
df['limite_credito'].fillna(mediana, inplace=True)
df['limite_credito'] = df['limite_credito'].astype(int)
df['limite_credito']

0         5000
1         6000
2            0
3         7000
4         7000
         ...  
22730    10000
22731     5000
22732     4750
22733     7000
22734     4750
Name: limite_credito, Length: 22735, dtype: int64

In [31]:
#Para rellenar nulos empleo leyenda "dato no disponible"
df.fillna("Dato no disponible", inplace=True)

In [32]:
nulos= df.isnull().sum()
nulos

folio                 0
tag                   0
folio_solicitud       0
fecha                 0
marca                 0
modelo                0
plazo                 0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
agente_venta          0
dis_venta             0
status                0
fraude                0
empresa               0
inversion             0
pagos_realizados      0
reautorizacion        0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
puntos                0
riesgo                0
score_buro            0
razones_buro          0
porc_eng              0
limite_credito        0
semana_actual         0
cp_cliente            0
edad_cliente          0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

In [33]:
#Tratamiento de outliers
for i in df.columns:
  if i != 'folio' and i != 'folio_solicitud':
    if df[i].dtypes == 'int64' or df[i].dtypes == 'float64':
      q1 = df[i].quantile(0.25)
      q3 = df[i].quantile(0.75)
      iqr = q3 - q1
      li = q1 - 1.5 * iqr
      ls = q3 + 1.5 * iqr
      df = df[(df[i] >= li) & (df[i] <= ls)]

In [34]:
df = df.drop(columns = [i for i in df.columns if df[i].dtypes == 'object'])

In [35]:
res = pd.DataFrame(columns=['Variable', 'Precisión', 'Exactitud', 'Puntaje F1'])
df1 = df.copy()
q50 = df['score_buro'].quantile(0.5)

def clasificacion(valor):
    if valor <= q50:
        return 'bajo'
    else:
        return 'alto'

df1['score_buro'] = df1['score_buro'].apply(clasificacion)

In [36]:
y = df1['score_buro']
X = df1[['precio',
        'costo_total',
        'enganche',
        'monto_financiado',
        'monto_accesorios',
        'pagos_realizados',
        'reautorizacion',
        'limite_credito',
        'descuento']]

In [37]:
#Modelo
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state =7)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [38]:
y_pred = algoritmo.predict(X_test)

matconf = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="binary", pos_label="alto")
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")

var = 'Todas'

res = res.append({'Variable': var,'Precisión': precision,'Exactitud': acc,'Puntaje F1': f1},ignore_index=True)

print('Matriz de Confusión:',matconf)
print('\nPrecisión:',precision)
print('\Accuracy:',acc)
print('\nPuntaje F1:',f1)


Matriz de Confusión: [[716 237]
 [120 835]]

Precisión: 0.8564593301435407
\Accuracy: 0.8128930817610063

Puntaje F1: 0.8004471771939632


<ipython-input-38-806e54014c9d>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': var,


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

c = 1

for i in X.columns:

  escalar = StandardScaler()
  X_train_i = X_train[[i]]
  X_test_i = X_test[[i]]
  X_train_i = escalar.fit_transform(X_train_i)
  X_test_i = escalar.transform(X_test_i)
  algoritmo = LogisticRegression()
  algoritmo.fit(X_train_i, y_train)
  y_pred = algoritmo.predict(X_test_i)

  matconf = confusion_matrix(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average="binary", pos_label="alto")
  acc = accuracy_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")


  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)

  print('________________________________')
  print(f'\n {c}. Variable {i} con score_buro')
  print('Matriz de Confusión:',matconf)
  print('\nPrecisión:',precision)
  print('\Accuracy:',acc)
  print('\nPuntaje F1:',f1)

  c += 1

res

<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)


________________________________

 1. Variable precio con score_buro
Matriz de Confusión: [[194 777]
 [172 765]]

Precisión: 0.5300546448087432
\Accuracy: 0.5026205450733753

Puntaje F1: 0.29020194465220644
________________________________

 2. Variable costo_total con score_buro
Matriz de Confusión: [[410 561]
 [355 582]]

Precisión: 0.5359477124183006
\Accuracy: 0.519916142557652

Puntaje F1: 0.47235023041474655


<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)
<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)


________________________________

 3. Variable enganche con score_buro
Matriz de Confusión: [[657 314]
 [403 534]]

Precisión: 0.619811320754717
\Accuracy: 0.6242138364779874

Puntaje F1: 0.6469719350073855
________________________________

 4. Variable monto_financiado con score_buro
Matriz de Confusión: [[388 583]
 [345 592]]

Precisión: 0.529331514324693
\Accuracy: 0.5136268343815513

Puntaje F1: 0.45539906103286387


<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)


...

<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)


________________________________

 6. Variable pagos_realizados con score_buro
Matriz de Confusión: [[173 798]
 [190 747]]

Precisión: 0.4765840220385675
\Accuracy: 0.48218029350104824

Puntaje F1: 0.25937031484257866
________________________________

 7. Variable reautorizacion con score_buro
Matriz de Confusión: [[  0 971]
 [  0 937]]

Precisión: 0.0
\Accuracy: 0.4910901467505241

Puntaje F1: 0.0


<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)
<ipython-input-40-f05d76765f58>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': i,'Precisión': precision, 'Accuracy': acc,'F1': f1},ignore_index=True)


________________________________

 8. Variable limite_credito con score_buro
Matriz de Confusión: [[671 300]
 [100 837]]

Precisión: 0.8702983138780804
\Accuracy: 0.790356394129979

Puntaje F1: 0.7703788748564868
________________________________

 9. Variable descuento con score_buro
Matriz de Confusión: [[267 704]
 [249 688]]

Precisión: 0.5174418604651163
\Accuracy: 0.500524109014675

Puntaje F1: 0.35911230665770005


,Variable,Precisión,Exactitud,Puntaje F1,Accuracy,F1
0,Todas,0.856459,0.812893,0.800447,NaN,NaN
1,precio,0.477825,NaN,NaN,0.480084,0.402410
2,costo_total,0.517491,NaN,NaN,0.511530,0.479330
3,enganche,0.625245,NaN,NaN,0.630503,0.644478
4,monto_financiado,0.529860,NaN,NaN,0.520964,0.477117
5,monto_accesorios,0.000000,NaN,NaN,0.496331,0.000000
6,pagos_realizados,0.000000,NaN,NaN,0.496331,0.000000
7,reautorizacion,0.000000,NaN,NaN,0.496331,0.000000
8,limite_credito,0.858586,NaN,NaN,0.794025,0.775813
9,descuento,0.551402,NaN,NaN,0.525157,0.394385
